# Ingeniería de características

### Importación de Librerías y Configuración Inicial

In [10]:
import pandas as pd
import numpy as np
import warnings
import os
import re
from pathlib import Path
import json
from datetime import datetime
import pandas as pd
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad
import base64
from dotenv import load_dotenv

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Cargar variables de entorno desde el archivo .env
load_dotenv('../../.env')

True

### Lectura de los datos limpios

In [11]:
# Carga de datos
file_path = '../cleaned_data/salud_mental_cleaned.csv'
df = pd.read_csv(file_path)

### Concatenación de los nombres

In [12]:
file_path = '../raw_data/column_names.csv'
nombres = pd.read_csv(file_path)

In [13]:
df['nombre_completo'] = nombres

## Cifrado de los nombres

In [14]:
# Leer la clave de cifrado desde las variables de entorno
encryption_key_str = os.getenv('ENCRYPTION_KEY')
if encryption_key_str is None:
    raise ValueError("No se encontró la clave ENCRYPTION_KEY en las variables de entorno")

# Convertir la clave string a bytes
key = encryption_key_str.encode('utf-8')

print(f"Clave de cifrado cargada exitosamente desde .env")
print(f"Longitud de la clave: {len(key)} bytes")

def encrypt_column(df, column_name, key):
    """
    Cifra una columna específica del DataFrame usando AES-GCM
    
    Args:
        df: DataFrame de pandas
        column_name: Nombre de la columna a cifrar (string)
        key: Clave de cifrado en bytes
    
    Returns:
        DataFrame con la columna cifrada
    """
    if column_name not in df.columns:
        raise ValueError(f"La columna '{column_name}' no existe en el DataFrame")
    
    encrypted_values = []

    for value in df[column_name]:
        if pd.isna(value):
            encrypted_values.append(None)
            continue

        nonce = get_random_bytes(12)  # 96 bits recomendado para GCM
        cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
        ciphertext, tag = cipher.encrypt_and_digest(str(value).encode("utf-8"))

        # Guardamos nonce + tag + ciphertext concatenados y codificados en base64
        encrypted = base64.b64encode(nonce + tag + ciphertext).decode("utf-8")
        encrypted_values.append(encrypted)

    # Crear una copia del DataFrame y actualizar la columna
    df_copy = df.copy()
    df_copy[column_name] = encrypted_values
    return df_copy

Clave de cifrado cargada exitosamente desde .env
Longitud de la clave: 32 bytes


In [15]:
df = encrypt_column(df, 'nombre_completo', key)

print("Función de cifrado lista para usar.")
print("Ejemplo de uso: df = encrypt_column(df, 'nombre_de_columna', key)")
print(f"Columnas disponibles en el DataFrame: {list(df.columns)}")

Función de cifrado lista para usar.
Ejemplo de uso: df = encrypt_column(df, 'nombre_de_columna', key)
Columnas disponibles en el DataFrame: ['comunidad_autonoma', 'fecha_nacimiento', 'sexo', 'fecha_ingreso', 'circunstancia_contacto', 'fecha_fin_contacto', 'tipo_alta', 'estancia_dias', 'diagnostico_principal', 'categoria', 'diagnostico_2', 'diagnostico_3', 'diagnostico_4', 'diagnostico_5', 'diagnostico_6', 'diagnostico_7', 'diagnostico_8', 'diagnostico_9', 'diagnostico_10', 'diagnostico_11', 'diagnostico_12', 'diagnostico_13', 'diagnostico_14', 'fecha_intervencion', 'procedimiento_1', 'procedimiento_2', 'procedimiento_3', 'procedimiento_4', 'procedimiento_5', 'procedimiento_6', 'procedimiento_7', 'procedimiento_8', 'procedimiento_9', 'procedimiento_10', 'procedimiento_11', 'procedimiento_12', 'procedimiento_13', 'procedimiento_14', 'procedimiento_15', 'procedimiento_16', 'procedimiento_17', 'procedimiento_18', 'procedimiento_19', 'procedimiento_20', 'grd_apr', 'cdm_apr', 'nivel_severida

In [16]:
df.head()

,comunidad_autonoma,fecha_nacimiento,sexo,fecha_ingreso,circunstancia_contacto,fecha_fin_contacto,tipo_alta,estancia_dias,diagnostico_principal,categoria,diagnostico_2,diagnostico_3,diagnostico_4,diagnostico_5,diagnostico_6,diagnostico_7,diagnostico_8,diagnostico_9,diagnostico_10,diagnostico_11,diagnostico_12,diagnostico_13,diagnostico_14,fecha_intervencion,procedimiento_1,procedimiento_2,procedimiento_3,procedimiento_4,procedimiento_5,procedimiento_6,procedimiento_7,procedimiento_8,procedimiento_9,procedimiento_10,procedimiento_11,procedimiento_12,procedimiento_13,procedimiento_14,procedimiento_15,procedimiento_16,procedimiento_17,procedimiento_18,procedimiento_19,procedimiento_20,grd_apr,cdm_apr,nivel_severidad_apr,riesgo_mortalidad_apr,servicio,edad,coste_apr,cie,numero_registro_anual,centro_recodificado,cip_sns_recodificado,pais_nacimiento,pais_residencia,fecha_inicio_contacto,regimen_financiacion,procedencia,continuidad_asistencial,ingreso_en_uci,dias_uci,diagnostico_15,diagnostico_16,diagnostico_17,diagnostico_18,diagnostico_19,diagnostico_20,poa_diagnostico_principal,poa_diagnostico_2,poa_diagnostico_3,poa_diagnostico_4,poa_diagnostico_5,poa_diagnostico_6,poa_diagnostico_7,poa_diagnostico_8,poa_diagnostico_9,poa_diagnostico_10,poa_diagnostico_11,poa_diagnostico_12,poa_diagnostico_13,poa_diagnostico_14,poa_diagnostico_15,poa_diagnostico_16,poa_diagnostico_17,poa_diagnostico_18,poa_diagnostico_19,poa_diagnostico_20,procedimiento_externo_1,procedimiento_externo_2,procedimiento_externo_3,tipo_grd_apr,peso_espanol_apr,edad_en_ingreso,mes_ingreso,tipo_alta_desc,nombre_completo
0,andalucia,1951-08-17,mujer,2016-01-01,1,2016-01-08,1,7,F25.0,"Esquizofrenia, trastornos esquizotípicos y tra...",Z63.79,Z91.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750,19,2,1,PSQ,74,6340,10,8537155.0,-2088791444897189888,109457269-593755146,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,E,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,1.393611,64,1,domicilio,RTuJOU0loRhD8KOqgyOdjanDkAGWxHuqTppZ8QdmrdGFvB...
1,andalucia,1929-03-20,mujer,2016-01-01,1,2016-01-08,1,7,F41.9,"Trastornos neuróticos, trastornos relacionados...",I11.9,I35.8,E11.9,I87.2,Z95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4B02XSZ,B246ZZZ,4A02X4Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,756,19,1,2,CAR,96,2771,10,8992115.0,-1166333372325380096,-1589750168781380096,desconocido,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,S,S,S,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,0.609264,86,1,domicilio,8g2YyBbLWee2xebZ/B2++1b9q5h2ZwS7BlTyw3Y+7NLtzO...
2,andalucia,1976-11-25,varon,2016-01-01,1,2016-01-11,1,10,F60.2,Trastornos de la personalidad y del comportami...,F19.288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,752,19,2,1,PSQ,48,4009,10,8998349.0,17490445801063320188,-5406560181117020160,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,0.881297,39,1,domicilio,MnUg1yFF6mijVcFJ5jVI3tpxdzBANJqSG2jIaWn365csna...
3,andalucia,1976-11-10,mujer,2016-01-01,1,2016-01-27,1,26,F20.0,"Esquizofrenia, trastornos esquizotípicos y tra...",C07,F17.210,F12.20,F14.10,F10.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750,19,1,2,PSQ,48,6073,10,8800205.0,-3960068041784730112,-1823171082,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,S,S,S,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,1.335036,39,1,domicilio,2WeQLWJGQLdJjg06qiIy7GYXogwz+g++XqU6/RVmrIuyy2...
4,andalucia,1977-04-28,mujer,2016-01-01,1,2016-01-18,1,17,F60.1,Trastornos de la personalidad y del comportami...,Z88.0,N

### Visualización inicial de los datos

In [17]:
df.head()

,comunidad_autonoma,fecha_nacimiento,sexo,fecha_ingreso,circunstancia_contacto,fecha_fin_contacto,tipo_alta,estancia_dias,diagnostico_principal,categoria,diagnostico_2,diagnostico_3,diagnostico_4,diagnostico_5,diagnostico_6,diagnostico_7,diagnostico_8,diagnostico_9,diagnostico_10,diagnostico_11,diagnostico_12,diagnostico_13,diagnostico_14,fecha_intervencion,procedimiento_1,procedimiento_2,procedimiento_3,procedimiento_4,procedimiento_5,procedimiento_6,procedimiento_7,procedimiento_8,procedimiento_9,procedimiento_10,procedimiento_11,procedimiento_12,procedimiento_13,procedimiento_14,procedimiento_15,procedimiento_16,procedimiento_17,procedimiento_18,procedimiento_19,procedimiento_20,grd_apr,cdm_apr,nivel_severidad_apr,riesgo_mortalidad_apr,servicio,edad,coste_apr,cie,numero_registro_anual,centro_recodificado,cip_sns_recodificado,pais_nacimiento,pais_residencia,fecha_inicio_contacto,regimen_financiacion,procedencia,continuidad_asistencial,ingreso_en_uci,dias_uci,diagnostico_15,diagnostico_16,diagnostico_17,diagnostico_18,diagnostico_19,diagnostico_20,poa_diagnostico_principal,poa_diagnostico_2,poa_diagnostico_3,poa_diagnostico_4,poa_diagnostico_5,poa_diagnostico_6,poa_diagnostico_7,poa_diagnostico_8,poa_diagnostico_9,poa_diagnostico_10,poa_diagnostico_11,poa_diagnostico_12,poa_diagnostico_13,poa_diagnostico_14,poa_diagnostico_15,poa_diagnostico_16,poa_diagnostico_17,poa_diagnostico_18,poa_diagnostico_19,poa_diagnostico_20,procedimiento_externo_1,procedimiento_externo_2,procedimiento_externo_3,tipo_grd_apr,peso_espanol_apr,edad_en_ingreso,mes_ingreso,tipo_alta_desc,nombre_completo
0,andalucia,1951-08-17,mujer,2016-01-01,1,2016-01-08,1,7,F25.0,"Esquizofrenia, trastornos esquizotípicos y tra...",Z63.79,Z91.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750,19,2,1,PSQ,74,6340,10,8537155.0,-2088791444897189888,109457269-593755146,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,E,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,1.393611,64,1,domicilio,RTuJOU0loRhD8KOqgyOdjanDkAGWxHuqTppZ8QdmrdGFvB...
1,andalucia,1929-03-20,mujer,2016-01-01,1,2016-01-08,1,7,F41.9,"Trastornos neuróticos, trastornos relacionados...",I11.9,I35.8,E11.9,I87.2,Z95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4B02XSZ,B246ZZZ,4A02X4Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,756,19,1,2,CAR,96,2771,10,8992115.0,-1166333372325380096,-1589750168781380096,desconocido,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,S,S,S,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,0.609264,86,1,domicilio,8g2YyBbLWee2xebZ/B2++1b9q5h2ZwS7BlTyw3Y+7NLtzO...
2,andalucia,1976-11-25,varon,2016-01-01,1,2016-01-11,1,10,F60.2,Trastornos de la personalidad y del comportami...,F19.288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,752,19,2,1,PSQ,48,4009,10,8998349.0,17490445801063320188,-5406560181117020160,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,0.881297,39,1,domicilio,MnUg1yFF6mijVcFJ5jVI3tpxdzBANJqSG2jIaWn365csna...
3,andalucia,1976-11-10,mujer,2016-01-01,1,2016-01-27,1,26,F20.0,"Esquizofrenia, trastornos esquizotípicos y tra...",C07,F17.210,F12.20,F14.10,F10.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750,19,1,2,PSQ,48,6073,10,8800205.0,-3960068041784730112,-1823171082,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,S,S,S,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,1.335036,39,1,domicilio,2WeQLWJGQLdJjg06qiIy7GYXogwz+g++XqU6/RVmrIuyy2...
4,andalucia,1977-04-28,mujer,2016-01-01,1,2016-01-18,1,17,F60.1,Trastornos de la personalidad y del comportami...,Z88.0,N

### Creación de la columna menor_edad (vale uno si es menor de edad, cero en caso contrario)

In [18]:
df['menor_edad'] = (df['edad'] < 18).astype(int)

print(f"\nColumna menor_edad creada:")
print(f"Distribución de menor_edad:")
print(df['menor_edad'].value_counts())
print(f"\nPorcentaje de menores de edad: {df['menor_edad'].mean()*100:.2f}%")

# Verificar algunos casos
print(f"\nEjemplos de menores de edad (edad < 18):")
menores = df[df['menor_edad'] == 1][['edad', 'menor_edad']].head()
if len(menores) > 0:
    print(menores)
else:
    print("No hay menores de edad en el dataset")

print(f"\nEjemplos de mayores de edad (edad >= 18):")
print(df[df['menor_edad'] == 0][['edad', 'menor_edad']].head())


Columna menor_edad creada:
Distribución de menor_edad:
menor_edad
0    21140
1       70
Name: count, dtype: int64

Porcentaje de menores de edad: 0.33%

Ejemplos de menores de edad (edad < 18):
      edad  menor_edad
799     14           1
926     15           1
2512    13           1
2516    17           1
2586    15           1

Ejemplos de mayores de edad (edad >= 18):
   edad  menor_edad
0    74           0
1    96           0
2    48           0
3    48           0
4    48           0


### Creación de la columna de si es español (uno si es español, cero en caso contrario)

In [19]:
df['nacido_espana'] = (df['pais_nacimiento'] == 'espana').astype(int)

print(f"\nColumna nacido_espana creada:")
print(f"Distribución de nacido_espana:")
print(df['nacido_espana'].value_counts())
print(f"\nPorcentaje de nacidos en España: {df['nacido_espana'].mean()*100:.2f}%")

# Verificar algunos casos
print(f"\nEjemplos de nacidos en España:")
print(df[df['nacido_espana'] == 1][['pais_nacimiento', 'nacido_espana']].head())

print(f"\nEjemplos de NO nacidos en España:")
print(df[df['nacido_espana'] == 0][['pais_nacimiento', 'nacido_espana']].head())


Columna nacido_espana creada:
Distribución de nacido_espana:
nacido_espana
1    16024
0     5186
Name: count, dtype: int64

Porcentaje de nacidos en España: 75.55%

Ejemplos de nacidos en España:
  pais_nacimiento  nacido_espana
0          espana              1
2          espana              1
3          espana              1
4          espana              1
5          espana              1

Ejemplos de NO nacidos en España:
   pais_nacimiento  nacido_espana
1      desconocido              0
10     desconocido              0
19     desconocido              0
21     desconocido              0
22        alemania              0


### Creación de la columna de si tiene una estancia larga (uno si la estancia es mayor o igual a 7 días, 0 en caso contrario)

In [20]:
df['larga_estancia'] = (df['estancia_dias'] >= 7).astype(int)

print(f"\nColumna larga_estancia creada:")
print(f"Distribución de larga_estancia:")
print(df['larga_estancia'].value_counts())
print(f"\nPorcentaje de estancias largas (≥7 días): {df['larga_estancia'].mean()*100:.2f}%")

# Verificar algunos casos
print(f"\nEjemplos de estancias largas (≥7 días):")
print(df[df['larga_estancia'] == 1][['estancia_dias', 'larga_estancia']].head())

print(f"\nEjemplos de estancias cortas (<7 días):")
print(df[df['larga_estancia'] == 0][['estancia_dias', 'larga_estancia']].head())

# Análisis adicional
print(f"\nAnálisis de estancias por categoría:")
print(f"Estancia promedio en casos de estancia corta: {df[df['larga_estancia']==0]['estancia_dias'].mean():.2f} días")
print(f"Estancia promedio en casos de estancia larga: {df[df['larga_estancia']==1]['estancia_dias'].mean():.2f} días")


Columna larga_estancia creada:
Distribución de larga_estancia:
larga_estancia
1    14786
0     6424
Name: count, dtype: int64

Porcentaje de estancias largas (≥7 días): 69.71%

Ejemplos de estancias largas (≥7 días):
   estancia_dias  larga_estancia
0              7               1
1              7               1
2             10               1
3             26               1
4             17               1

Ejemplos de estancias cortas (<7 días):
    estancia_dias  larga_estancia
6               3               0
9               1               0
11              5               0
12              6               0
14              3               0

Análisis de estancias por categoría:
Estancia promedio en casos de estancia corta: 3.47 días
Estancia promedio en casos de estancia larga: 20.67 días


### Dummy enconding de la variable categoría

Estudiamos los valores que toma:

In [21]:
df['categoria'].unique()

array(['Esquizofrenia, trastornos esquizotípicos y trastornos delirantes',
       'Trastornos neuróticos, trastornos relacionados con el estrés y trastornos somatomorfos',
       'Trastornos de la personalidad y del comportamiento en adultos',
       'Trastornos mentales y del comportamiento debidos al uso de sustancias psicoactivas',
       'Trastornos del humor [afectivos]',
       'Trastornos emocionales y del comportamiento que aparecen habitualmente en la niñez y en la adolescencia',
       'Síndromes del comportamiento asociados con alteraciones fisiológicas y factores físicos'],
      dtype=object)

Creamos tantas columnas menos una, para evitar la multicolinealidad

In [22]:
# Verificar los valores únicos de la columna categoria
print("Valores únicos en la columna 'categoria':")
categorias_unicas = df['categoria'].unique()
for i, cat in enumerate(categorias_unicas):
    print(f"{i+1}. {cat}")

print(f"\nTotal de categorías: {len(categorias_unicas)}")
print(f"\nDistribución de categorías:")
print(df['categoria'].value_counts())

# Crear las columnas dummy según los nombres especificados
# Inicializar todas las columnas en 0
df['cat_esquizofrenia'] = 0
df['cat_trastorno_neurotico'] = 0
df['cat_trastorno_personalidad'] = 0
df['cat_trastorno_mental'] = 0
df['cat_trastorno_humor'] = 0
df['cat_trastorno_emocional'] = 0

# Asignar 1 según la categoría correspondiente
df.loc[df['categoria'] == 'Esquizofrenia, trastornos esquizotípicos y trastornos delirantes', 'cat_esquizofrenia'] = 1
df.loc[df['categoria'] == 'Trastornos neuróticos, trastornos relacionados con el estrés y trastornos somatomorfos', 'cat_trastorno_neurotico'] = 1
df.loc[df['categoria'] == 'Trastornos de la personalidad y del comportamiento en adultos', 'cat_trastorno_personalidad'] = 1
df.loc[df['categoria'] == 'Trastornos mentales y del comportamiento debidos al uso de sustancias psicoactivas', 'cat_trastorno_mental'] = 1
df.loc[df['categoria'] == 'Trastornos del humor [afectivos]', 'cat_trastorno_humor'] = 1
df.loc[df['categoria'] == 'Trastornos emocionales y del comportamiento que aparecen habitualmente en la niñez y en la adolescencia', 'cat_trastorno_emocional'] = 1

# Verificar que la suma de todas las columnas dummy es igual al número de filas
columnas_dummy = ['cat_esquizofrenia', 'cat_trastorno_neurotico', 'cat_trastorno_personalidad', 
                  'cat_trastorno_mental', 'cat_trastorno_humor', 'cat_trastorno_emocional']

suma_dummies = df[columnas_dummy].sum(axis=1)
print(f"\nVerificación del dummy encoding:")
print(f"Todas las filas tienen exactamente una categoría: {(suma_dummies == 1).all()}")
print(f"Distribución de la suma por fila:")
print(suma_dummies.value_counts())


Valores únicos en la columna 'categoria':
1. Esquizofrenia, trastornos esquizotípicos y trastornos delirantes
2. Trastornos neuróticos, trastornos relacionados con el estrés y trastornos somatomorfos
3. Trastornos de la personalidad y del comportamiento en adultos
4. Trastornos mentales y del comportamiento debidos al uso de sustancias psicoactivas
5. Trastornos del humor [afectivos]
6. Trastornos emocionales y del comportamiento que aparecen habitualmente en la niñez y en la adolescencia
7. Síndromes del comportamiento asociados con alteraciones fisiológicas y factores físicos

Total de categorías: 7

Distribución de categorías:
categoria
Esquizofrenia, trastornos esquizotípicos y trastornos delirantes                                           9126
Trastornos del humor [afectivos]                                                                           5224
Trastornos de la personalidad y del comportamiento en adultos                                              3248
Trastornos neuró

In [23]:
df.head()

,comunidad_autonoma,fecha_nacimiento,sexo,fecha_ingreso,circunstancia_contacto,fecha_fin_contacto,tipo_alta,estancia_dias,diagnostico_principal,categoria,diagnostico_2,diagnostico_3,diagnostico_4,diagnostico_5,diagnostico_6,diagnostico_7,diagnostico_8,diagnostico_9,diagnostico_10,diagnostico_11,diagnostico_12,diagnostico_13,diagnostico_14,fecha_intervencion,procedimiento_1,procedimiento_2,procedimiento_3,procedimiento_4,procedimiento_5,procedimiento_6,procedimiento_7,procedimiento_8,procedimiento_9,procedimiento_10,procedimiento_11,procedimiento_12,procedimiento_13,procedimiento_14,procedimiento_15,procedimiento_16,procedimiento_17,procedimiento_18,procedimiento_19,procedimiento_20,grd_apr,cdm_apr,nivel_severidad_apr,riesgo_mortalidad_apr,servicio,edad,coste_apr,cie,numero_registro_anual,centro_recodificado,cip_sns_recodificado,pais_nacimiento,pais_residencia,fecha_inicio_contacto,regimen_financiacion,procedencia,continuidad_asistencial,ingreso_en_uci,dias_uci,diagnostico_15,diagnostico_16,diagnostico_17,diagnostico_18,diagnostico_19,diagnostico_20,poa_diagnostico_principal,poa_diagnostico_2,poa_diagnostico_3,poa_diagnostico_4,poa_diagnostico_5,poa_diagnostico_6,poa_diagnostico_7,poa_diagnostico_8,poa_diagnostico_9,poa_diagnostico_10,poa_diagnostico_11,poa_diagnostico_12,poa_diagnostico_13,poa_diagnostico_14,poa_diagnostico_15,poa_diagnostico_16,poa_diagnostico_17,poa_diagnostico_18,poa_diagnostico_19,poa_diagnostico_20,procedimiento_externo_1,procedimiento_externo_2,procedimiento_externo_3,tipo_grd_apr,peso_espanol_apr,edad_en_ingreso,mes_ingreso,tipo_alta_desc,nombre_completo,menor_edad,nacido_espana,larga_estancia,cat_esquizofrenia,cat_trastorno_neurotico,cat_trastorno_personalidad,cat_trastorno_mental,cat_trastorno_humor,cat_trastorno_emocional
0,andalucia,1951-08-17,mujer,2016-01-01,1,2016-01-08,1,7,F25.0,"Esquizofrenia, trastornos esquizotípicos y tra...",Z63.79,Z91.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750,19,2,1,PSQ,74,6340,10,8537155.0,-2088791444897189888,109457269-593755146,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,E,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,1.393611,64,1,domicilio,RTuJOU0loRhD8KOqgyOdjanDkAGWxHuqTppZ8QdmrdGFvB...,0,1,1,1,0,0,0,0,0
1,andalucia,1929-03-20,mujer,2016-01-01,1,2016-01-08,1,7,F41.9,"Trastornos neuróticos, trastornos relacionados...",I11.9,I35.8,E11.9,I87.2,Z95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4B02XSZ,B246ZZZ,4A02X4Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,756,19,1,2,CAR,96,2771,10,8992115.0,-1166333372325380096,-1589750168781380096,desconocido,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,S,S,S,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,0.609264,86,1,domicilio,8g2YyBbLWee2xebZ/B2++1b9q5h2ZwS7BlTyw3Y+7NLtzO...,0,0,1,0,1,0,0,0,0
2,andalucia,1976-11-25,varon,2016-01-01,1,2016-01-11,1,10,F60.2,Trastornos de la personalidad y del comportami...,F19.288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,752,19,2,1,PSQ,48,4009,10,8998349.0,17490445801063320188,-5406560181117020160,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,0.881297,39,1,domicilio,MnUg1yFF6mijVcFJ5jVI3tpxdzBANJqSG2jIaWn365csna...,0,1,1,0,0,1,0,0,0
3,andalucia,1976-11-10,mujer,2016-01-01,1,2016-01-27,1,26,F20.0,"Esquizofrenia, trastornos esquizotípicos y tra...",C07,F17.210,F12.20,F14.10,F10.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750,19,1,2,PSQ,48,6073,10,8800205.0,-3960068041784730112,-1823171082,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,S,S,S,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

### Creación de columnas que cuentan diagnósticos secundarios y procedimientos no nulos

In [24]:
# Columnas de diagnósticos secundarios (excluir diagnostico_principal)
columnas_diagnosticos_secundarios = [col for col in df.columns if col.startswith('diagnostico_') and col != 'diagnostico_principal']
print(f"Columnas de diagnósticos secundarios encontradas ({len(columnas_diagnosticos_secundarios)}):")
for i, col in enumerate(columnas_diagnosticos_secundarios, 1):
    print(f"  {i:2d}. {col}")

# Columnas de procedimientos (incluir procedimiento_1 a procedimiento_20, pero excluir procedimiento_externo)
columnas_procedimientos = [col for col in df.columns if col.startswith('procedimiento_') and not col.startswith('procedimiento_externo')]
print(f"\nColumnas de procedimientos encontradas ({len(columnas_procedimientos)}):")
for i, col in enumerate(columnas_procedimientos, 1):
    print(f"  {i:2d}. {col}")

# Crear la columna num_diagnosticos_secundarios
df['num_diagnosticos_secundarios'] = df[columnas_diagnosticos_secundarios].notna().sum(axis=1)

print(f"\n\nCOLUMNA num_diagnosticos_secundarios CREADA")
print("="*50)
print(f"Estadísticas de num_diagnosticos_secundarios:")
print(df['num_diagnosticos_secundarios'].describe())
print(f"\nDistribución de número de diagnósticos secundarios:")
print(df['num_diagnosticos_secundarios'].value_counts().sort_index().head(15))

# Crear la columna num_procedimientos
df['num_procedimientos'] = df[columnas_procedimientos].notna().sum(axis=1)

print(f"\n\nCOLUMNA num_procedimientos CREADA")
print("="*50)
print(f"Estadísticas de num_procedimientos:")
print(df['num_procedimientos'].describe())
print(f"\nDistribución de número de procedimientos:")
print(df['num_procedimientos'].value_counts().sort_index().head(15))

# Análisis combinado
print(f"\n\nANÁLISIS COMBINADO")
print("="*40)
print(f"Promedio de diagnósticos secundarios por paciente: {df['num_diagnosticos_secundarios'].mean():.2f}")
print(f"Promedio de procedimientos por paciente: {df['num_procedimientos'].mean():.2f}")

# Verificar algunos ejemplos
print(f"\nEjemplos de pacientes con más diagnósticos secundarios:")
ejemplos_diag = df.nlargest(5, 'num_diagnosticos_secundarios')[['diagnostico_principal', 'num_diagnosticos_secundarios', 'num_procedimientos']]
print(ejemplos_diag)

print(f"\nEjemplos de pacientes con más procedimientos:")
ejemplos_proc = df.nlargest(5, 'num_procedimientos')[['diagnostico_principal', 'num_diagnosticos_secundarios', 'num_procedimientos']]
print(ejemplos_proc)

# Correlación entre ambas variables
correlacion = df['num_diagnosticos_secundarios'].corr(df['num_procedimientos'])
print(f"\nCorrelación entre diagnósticos secundarios y procedimientos: {correlacion:.3f}")

Columnas de diagnósticos secundarios encontradas (19):
   1. diagnostico_2
   2. diagnostico_3
   3. diagnostico_4
   4. diagnostico_5
   5. diagnostico_6
   6. diagnostico_7
   7. diagnostico_8
   8. diagnostico_9
   9. diagnostico_10
  10. diagnostico_11
  11. diagnostico_12
  12. diagnostico_13
  13. diagnostico_14
  14. diagnostico_15
  15. diagnostico_16
  16. diagnostico_17
  17. diagnostico_18
  18. diagnostico_19
  19. diagnostico_20

Columnas de procedimientos encontradas (20):
   1. procedimiento_1
   2. procedimiento_2
   3. procedimiento_3
   4. procedimiento_4
   5. procedimiento_5
   6. procedimiento_6
   7. procedimiento_7
   8. procedimiento_8
   9. procedimiento_9
  10. procedimiento_10
  11. procedimiento_11
  12. procedimiento_12
  13. procedimiento_13
  14. procedimiento_14
  15. procedimiento_15
  16. procedimiento_16
  17. procedimiento_17
  18. procedimiento_18
  19. procedimiento_19
  20. procedimiento_20


COLUMNA num_diagnosticos_secundarios CREADA
Estadística

In [25]:
df.head()

,comunidad_autonoma,fecha_nacimiento,sexo,fecha_ingreso,circunstancia_contacto,fecha_fin_contacto,tipo_alta,estancia_dias,diagnostico_principal,categoria,diagnostico_2,diagnostico_3,diagnostico_4,diagnostico_5,diagnostico_6,diagnostico_7,diagnostico_8,diagnostico_9,diagnostico_10,diagnostico_11,diagnostico_12,diagnostico_13,diagnostico_14,fecha_intervencion,procedimiento_1,procedimiento_2,procedimiento_3,procedimiento_4,procedimiento_5,procedimiento_6,procedimiento_7,procedimiento_8,procedimiento_9,procedimiento_10,procedimiento_11,procedimiento_12,procedimiento_13,procedimiento_14,procedimiento_15,procedimiento_16,procedimiento_17,procedimiento_18,procedimiento_19,procedimiento_20,grd_apr,cdm_apr,nivel_severidad_apr,riesgo_mortalidad_apr,servicio,edad,coste_apr,cie,numero_registro_anual,centro_recodificado,cip_sns_recodificado,pais_nacimiento,pais_residencia,fecha_inicio_contacto,regimen_financiacion,procedencia,continuidad_asistencial,ingreso_en_uci,dias_uci,diagnostico_15,diagnostico_16,diagnostico_17,diagnostico_18,diagnostico_19,diagnostico_20,poa_diagnostico_principal,poa_diagnostico_2,poa_diagnostico_3,poa_diagnostico_4,poa_diagnostico_5,poa_diagnostico_6,poa_diagnostico_7,poa_diagnostico_8,poa_diagnostico_9,poa_diagnostico_10,poa_diagnostico_11,poa_diagnostico_12,poa_diagnostico_13,poa_diagnostico_14,poa_diagnostico_15,poa_diagnostico_16,poa_diagnostico_17,poa_diagnostico_18,poa_diagnostico_19,poa_diagnostico_20,procedimiento_externo_1,procedimiento_externo_2,procedimiento_externo_3,tipo_grd_apr,peso_espanol_apr,edad_en_ingreso,mes_ingreso,tipo_alta_desc,nombre_completo,menor_edad,nacido_espana,larga_estancia,cat_esquizofrenia,cat_trastorno_neurotico,cat_trastorno_personalidad,cat_trastorno_mental,cat_trastorno_humor,cat_trastorno_emocional,num_diagnosticos_secundarios,num_procedimientos
0,andalucia,1951-08-17,mujer,2016-01-01,1,2016-01-08,1,7,F25.0,"Esquizofrenia, trastornos esquizotípicos y tra...",Z63.79,Z91.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750,19,2,1,PSQ,74,6340,10,8537155.0,-2088791444897189888,109457269-593755146,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,E,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,1.393611,64,1,domicilio,RTuJOU0loRhD8KOqgyOdjanDkAGWxHuqTppZ8QdmrdGFvB...,0,1,1,1,0,0,0,0,0,2,0
1,andalucia,1929-03-20,mujer,2016-01-01,1,2016-01-08,1,7,F41.9,"Trastornos neuróticos, trastornos relacionados...",I11.9,I35.8,E11.9,I87.2,Z95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4B02XSZ,B246ZZZ,4A02X4Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,756,19,1,2,CAR,96,2771,10,8992115.0,-1166333372325380096,-1589750168781380096,desconocido,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,S,S,S,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,0.609264,86,1,domicilio,8g2YyBbLWee2xebZ/B2++1b9q5h2ZwS7BlTyw3Y+7NLtzO...,0,0,1,0,1,0,0,0,0,5,3
2,andalucia,1976-11-25,varon,2016-01-01,1,2016-01-11,1,10,F60.2,Trastornos de la personalidad y del comportami...,F19.288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,752,19,2,1,PSQ,48,4009,10,8998349.0,17490445801063320188,-5406560181117020160,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,0.881297,39,1,domicilio,MnUg1yFF6mijVcFJ5jVI3tpxdzBANJqSG2jIaWn365csna...,0,1,1,0,0,1,0,0,0,1,0
3,andalucia,1976-11-10,mujer,2016-01-01,1,2016-01-27,1,26,F20.0,"Esquizofrenia, trastornos esquizotípicos y tra...",C07,F17.210,F12.20,F14.10,F10.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750,19,1,2,PSQ,48,6073,10,8800205.0,-3960068041784730112,-1823171082,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,N

### Ideas más avanzadas:

### Complejidad del paciente

In [26]:
# Función para calcular complejidad del paciente
def calcular_complejidad(row):
    """
    Calcula un score de complejidad basado en múltiples factores clínicos
    """
    score = 0
    score += row['num_diagnosticos_secundarios'] * 0.3
    score += row['num_procedimientos'] * 0.2
    score += (row['nivel_severidad_apr'] - 1) * 0.25
    score += (row['riesgo_mortalidad_apr'] - 1) * 0.25
    return score

# Aplicar la función a cada fila para crear la columna complejidad_general
df['complejidad_general'] = df.apply(calcular_complejidad, axis=1)

# Estadísticas descriptivas de la nueva columna
print("Estadísticas de complejidad_general:")
print(df['complejidad_general'].describe())

print(f"\nDistribución de complejidad_general:")
print(df['complejidad_general'].value_counts().sort_index().head(20))

# Crear categorías de complejidad para mejor interpretación
def categorizar_complejidad(score):
    if score <= 1.0:
        return 'baja'
    elif score <= 2.5:
        return 'media'
    elif score <= 4.0:
        return 'alta'
    else:
        return 'muy alta'

df['categoria_complejidad'] = df['complejidad_general'].apply(categorizar_complejidad)

print(f"\nCategorías de complejidad creadas:")
print(df['categoria_complejidad'].value_counts())
print(f"\nPorcentajes por categoría:")
porcentajes = df['categoria_complejidad'].value_counts(normalize=True) * 100
for categoria, porcentaje in porcentajes.items():
    print(f"  {categoria}: {porcentaje:.1f}%")

# Análisis por componentes
print(f"\n\nANÁLISIS POR COMPONENTES")
print("="*40)
print(f"Contribución promedio por componente:")
print(f"  Diagnósticos secundarios: {(df['num_diagnosticos_secundarios'] * 0.3).mean():.3f}")
print(f"  Procedimientos: {(df['num_procedimientos'] * 0.2).mean():.3f}")
print(f"  Severidad APR: {((df['nivel_severidad_apr'] - 1) * 0.25).mean():.3f}")
print(f"  Riesgo mortalidad: {((df['riesgo_mortalidad_apr'] - 1) * 0.25).mean():.3f}")

# Ejemplos de casos de diferentes complejidades
print(f"\n\nEJEMPLOS POR CATEGORÍA DE COMPLEJIDAD")
print("="*50)
for categoria in ['baja', 'media', 'alta', 'muy alta']:
    ejemplos = df[df['categoria_complejidad'] == categoria].head(3)
    if len(ejemplos) > 0:
        print(f"\nComplejidad {categoria}:")
        cols_mostrar = ['complejidad_general', 'num_diagnosticos_secundarios', 'num_procedimientos', 
                       'nivel_severidad_apr', 'riesgo_mortalidad_apr', 'categoria']
        print(ejemplos[cols_mostrar])


# Análisis por categoría de diagnóstico
print(f"\nComplejidad promedio por categoría de diagnóstico:")
complejidad_por_categoria = df.groupby('categoria')['complejidad_general'].agg(['mean', 'std', 'count']).round(3)
print(complejidad_por_categoria)

Estadísticas de complejidad_general:
count    21210.000000
mean         1.226146
std          0.962730
min          0.000000
25%          0.550000
50%          1.050000
75%          1.750000
max          8.700000
Name: complejidad_general, dtype: float64

Distribución de complejidad_general:
complejidad_general
0.00    1820
0.20     139
0.25     370
0.30    1910
0.40     131
0.45      25
0.50     185
0.55     924
0.60    1601
0.60      26
0.65      44
0.70     214
0.75      96
0.80     178
0.85    1207
0.85      12
0.90    1268
0.90       1
0.90      55
0.95      78
Name: count, dtype: int64

Categorías de complejidad creadas:
categoria_complejidad
baja        10498
media        8707
alta         1691
muy alta      314
Name: count, dtype: int64

Porcentajes por categoría:
  baja: 49.5%
  media: 41.1%
  alta: 8.0%
  muy alta: 1.5%


ANÁLISIS POR COMPONENTES
Contribución promedio por componente:
  Diagnósticos secundarios: 0.989
  Procedimientos: 0.089
  Severidad APR: 0.134
  Riesgo mor

In [27]:
df.head()

,comunidad_autonoma,fecha_nacimiento,sexo,fecha_ingreso,circunstancia_contacto,fecha_fin_contacto,tipo_alta,estancia_dias,diagnostico_principal,categoria,diagnostico_2,diagnostico_3,diagnostico_4,diagnostico_5,diagnostico_6,diagnostico_7,diagnostico_8,diagnostico_9,diagnostico_10,diagnostico_11,diagnostico_12,diagnostico_13,diagnostico_14,fecha_intervencion,procedimiento_1,procedimiento_2,procedimiento_3,procedimiento_4,procedimiento_5,procedimiento_6,procedimiento_7,procedimiento_8,procedimiento_9,procedimiento_10,procedimiento_11,procedimiento_12,procedimiento_13,procedimiento_14,procedimiento_15,procedimiento_16,procedimiento_17,procedimiento_18,procedimiento_19,procedimiento_20,grd_apr,cdm_apr,nivel_severidad_apr,riesgo_mortalidad_apr,servicio,edad,coste_apr,cie,numero_registro_anual,centro_recodificado,cip_sns_recodificado,pais_nacimiento,pais_residencia,fecha_inicio_contacto,regimen_financiacion,procedencia,continuidad_asistencial,ingreso_en_uci,dias_uci,diagnostico_15,diagnostico_16,diagnostico_17,diagnostico_18,diagnostico_19,diagnostico_20,poa_diagnostico_principal,poa_diagnostico_2,poa_diagnostico_3,poa_diagnostico_4,poa_diagnostico_5,poa_diagnostico_6,poa_diagnostico_7,poa_diagnostico_8,poa_diagnostico_9,poa_diagnostico_10,poa_diagnostico_11,poa_diagnostico_12,poa_diagnostico_13,poa_diagnostico_14,poa_diagnostico_15,poa_diagnostico_16,poa_diagnostico_17,poa_diagnostico_18,poa_diagnostico_19,poa_diagnostico_20,procedimiento_externo_1,procedimiento_externo_2,procedimiento_externo_3,tipo_grd_apr,peso_espanol_apr,edad_en_ingreso,mes_ingreso,tipo_alta_desc,nombre_completo,menor_edad,nacido_espana,larga_estancia,cat_esquizofrenia,cat_trastorno_neurotico,cat_trastorno_personalidad,cat_trastorno_mental,cat_trastorno_humor,cat_trastorno_emocional,num_diagnosticos_secundarios,num_procedimientos,complejidad_general,categoria_complejidad
0,andalucia,1951-08-17,mujer,2016-01-01,1,2016-01-08,1,7,F25.0,"Esquizofrenia, trastornos esquizotípicos y tra...",Z63.79,Z91.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750,19,2,1,PSQ,74,6340,10,8537155.0,-2088791444897189888,109457269-593755146,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,E,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,1.393611,64,1,domicilio,RTuJOU0loRhD8KOqgyOdjanDkAGWxHuqTppZ8QdmrdGFvB...,0,1,1,1,0,0,0,0,0,2,0,0.85,baja
1,andalucia,1929-03-20,mujer,2016-01-01,1,2016-01-08,1,7,F41.9,"Trastornos neuróticos, trastornos relacionados...",I11.9,I35.8,E11.9,I87.2,Z95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4B02XSZ,B246ZZZ,4A02X4Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,756,19,1,2,CAR,96,2771,10,8992115.0,-1166333372325380096,-1589750168781380096,desconocido,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,S,S,S,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,0.609264,86,1,domicilio,8g2YyBbLWee2xebZ/B2++1b9q5h2ZwS7BlTyw3Y+7NLtzO...,0,0,1,0,1,0,0,0,0,5,3,2.35,media
2,andalucia,1976-11-25,varon,2016-01-01,1,2016-01-11,1,10,F60.2,Trastornos de la personalidad y del comportami...,F19.288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,752,19,2,1,PSQ,48,4009,10,8998349.0,17490445801063320188,-5406560181117020160,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,0.881297,39,1,domicilio,MnUg1yFF6mijVcFJ5jVI3tpxdzBANJqSG2jIaWn365csna...,0,1,1,0,0,1,0,0,0,1,0,0.55,baja
3,andalucia,1976-11-10,mujer,2016-01-01,1,2016-01-27,1,26,F20.0,"Esquizofrenia, trastornos esquizotípicos y tra...",C07,F17.210,F12.20,F14.10,F10.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750,19,1,2,PSQ,48,6073,10,8800205.0,-396006804178473011

### Columna de grupo_etario

In [28]:
# Función para asignar grupo etario basado en la edad
def asignar_grupo_etario(edad):
    """
    Asigna grupo etario especializado en salud mental basado en la edad
    """
    if edad >= 7 and edad <= 17:
        return 'pediatrico'
    elif edad >= 18 and edad <= 30:
        return 'joven adulto'
    elif edad >= 31 and edad <= 45:
        return 'adulto temprano'
    elif edad >= 46 and edad <= 60:
        return 'adulto medio'
    elif edad >= 61 and edad <= 75:
        return 'adulto mayor'
    elif edad >= 76:
        return 'geriatrico'
    else:
        return 'fuera de rango'  # Para edades < 7 años (si las hay)

# Crear la columna grupo_etario
df['grupo_etario'] = df['edad'].apply(asignar_grupo_etario)

print("COLUMNA grupo_etario CREADA")
print("="*50)

# Verificar el rango de edades en el dataset
print(f"Rango de edades en el dataset: {df['edad'].min()} - {df['edad'].max()} años")

# Distribución de grupos etarios
print(f"\nDistribución de grupos etarios:")
distribucion = df['grupo_etario'].value_counts()
print(distribucion)

print(f"\nPorcentajes por grupo etario:")
porcentajes = df['grupo_etario'].value_counts(normalize=True) * 100
for grupo, porcentaje in porcentajes.items():
    print(f"  {grupo}: {porcentaje:.1f}%")

# Estadísticas por grupo etario
print(f"\n\nESTADÍSTICAS POR GRUPO ETARIO")
print("="*50)
stats_por_grupo = df.groupby('grupo_etario')['edad'].agg(['min', 'max', 'mean', 'count']).round(1)
print(stats_por_grupo)


# Verificar si hay casos fuera de rango
casos_fuera_rango = len(df[df['grupo_etario'] == 'Fuera de rango'])
if casos_fuera_rango > 0:
    print(f"\n ATENCIÓN: {casos_fuera_rango} casos con edades fuera del rango esperado (< 7 años)")
    print("Edades fuera de rango:")
    print(df[df['grupo_etario'] == 'Fuera de rango']['edad'].value_counts())
else:
    print(f"\nTodos los casos están dentro de los rangos etarios definidos")

COLUMNA grupo_etario CREADA
Rango de edades en el dataset: 7 - 104 años

Distribución de grupos etarios:
grupo_etario
adulto medio       8320
adulto temprano    5557
adulto mayor       4894
joven adulto       1325
geriatrico         1044
pediatrico           70
Name: count, dtype: int64

Porcentajes por grupo etario:
  adulto medio: 39.2%
  adulto temprano: 26.2%
  adulto mayor: 23.1%
  joven adulto: 6.2%
  geriatrico: 4.9%
  pediatrico: 0.3%


ESTADÍSTICAS POR GRUPO ETARIO
                 min  max  mean  count
grupo_etario                          
adulto mayor      61   75  66.0   4894
adulto medio      46   60  53.0   8320
adulto temprano   31   45  38.9   5557
geriatrico        76  104  82.1   1044
joven adulto      18   30  26.2   1325
pediatrico         7   17  14.4     70

Todos los casos están dentro de los rangos etarios definidos


In [29]:
df.head()

,comunidad_autonoma,fecha_nacimiento,sexo,fecha_ingreso,circunstancia_contacto,fecha_fin_contacto,tipo_alta,estancia_dias,diagnostico_principal,categoria,diagnostico_2,diagnostico_3,diagnostico_4,diagnostico_5,diagnostico_6,diagnostico_7,diagnostico_8,diagnostico_9,diagnostico_10,diagnostico_11,diagnostico_12,diagnostico_13,diagnostico_14,fecha_intervencion,procedimiento_1,procedimiento_2,procedimiento_3,procedimiento_4,procedimiento_5,procedimiento_6,procedimiento_7,procedimiento_8,procedimiento_9,procedimiento_10,procedimiento_11,procedimiento_12,procedimiento_13,procedimiento_14,procedimiento_15,procedimiento_16,procedimiento_17,procedimiento_18,procedimiento_19,procedimiento_20,grd_apr,cdm_apr,nivel_severidad_apr,riesgo_mortalidad_apr,servicio,edad,coste_apr,cie,numero_registro_anual,centro_recodificado,cip_sns_recodificado,pais_nacimiento,pais_residencia,fecha_inicio_contacto,regimen_financiacion,procedencia,continuidad_asistencial,ingreso_en_uci,dias_uci,diagnostico_15,diagnostico_16,diagnostico_17,diagnostico_18,diagnostico_19,diagnostico_20,poa_diagnostico_principal,poa_diagnostico_2,poa_diagnostico_3,poa_diagnostico_4,poa_diagnostico_5,poa_diagnostico_6,poa_diagnostico_7,poa_diagnostico_8,poa_diagnostico_9,poa_diagnostico_10,poa_diagnostico_11,poa_diagnostico_12,poa_diagnostico_13,poa_diagnostico_14,poa_diagnostico_15,poa_diagnostico_16,poa_diagnostico_17,poa_diagnostico_18,poa_diagnostico_19,poa_diagnostico_20,procedimiento_externo_1,procedimiento_externo_2,procedimiento_externo_3,tipo_grd_apr,peso_espanol_apr,edad_en_ingreso,mes_ingreso,tipo_alta_desc,nombre_completo,menor_edad,nacido_espana,larga_estancia,cat_esquizofrenia,cat_trastorno_neurotico,cat_trastorno_personalidad,cat_trastorno_mental,cat_trastorno_humor,cat_trastorno_emocional,num_diagnosticos_secundarios,num_procedimientos,complejidad_general,categoria_complejidad,grupo_etario
0,andalucia,1951-08-17,mujer,2016-01-01,1,2016-01-08,1,7,F25.0,"Esquizofrenia, trastornos esquizotípicos y tra...",Z63.79,Z91.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750,19,2,1,PSQ,74,6340,10,8537155.0,-2088791444897189888,109457269-593755146,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,E,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,1.393611,64,1,domicilio,RTuJOU0loRhD8KOqgyOdjanDkAGWxHuqTppZ8QdmrdGFvB...,0,1,1,1,0,0,0,0,0,2,0,0.85,baja,adulto mayor
1,andalucia,1929-03-20,mujer,2016-01-01,1,2016-01-08,1,7,F41.9,"Trastornos neuróticos, trastornos relacionados...",I11.9,I35.8,E11.9,I87.2,Z95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4B02XSZ,B246ZZZ,4A02X4Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,756,19,1,2,CAR,96,2771,10,8992115.0,-1166333372325380096,-1589750168781380096,desconocido,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,S,S,S,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,0.609264,86,1,domicilio,8g2YyBbLWee2xebZ/B2++1b9q5h2ZwS7BlTyw3Y+7NLtzO...,0,0,1,0,1,0,0,0,0,5,3,2.35,media,geriatrico
2,andalucia,1976-11-25,varon,2016-01-01,1,2016-01-11,1,10,F60.2,Trastornos de la personalidad y del comportami...,F19.288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,752,19,2,1,PSQ,48,4009,10,8998349.0,17490445801063320188,-5406560181117020160,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,0.881297,39,1,domicilio,MnUg1yFF6mijVcFJ5jVI3tpxdzBANJqSG2jIaWn365csna...,0,1,1,0,0,1,0,0,0,1,0,0.55,baja,adulto medio
3,andalucia,1976-11-10,mujer,2016-01-01,1,2016-01-27,1,26,F20.0,"Esquizofrenia, trastornos esquizotípicos y tra...",C07,F17.210,F12.20,F14.10,F10.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750,1

### Guardado del dataset

In [30]:
# Guardar el dataset con las nuevas features
output_path = '../cleaned_data/salud_mental_featured.csv'
df.to_csv(output_path, index=False)
